# Грузим библиотеки

In [ ]:
from flask import Flask
from flask import request
import requests
from flask import make_response
import os
import json
from pandas import DataFrame
import traceback


from methods import access
from methods.emoji import emoji
from methods import telegram_bot_methods
from methods import google_maps_methods
from methods import psql_methods
from methods import reply
from dicts import dict_dates



import app



if __name__ == "__main__":

    
    app.application.run()



In [ ]:
import re

task_name = ' ! fdfs fdsfsdf fds'
is_main_task = task_name.replace(' ','')[0] == "!"
if is_main_task

In [ ]:
X = {'update_id': 201011794, 'message': {'message_id': 159, 'from': {'id': 84723474, 'first_name': 'Александр', 'last_name': 'Толмачёв', 'language_code': 'ru-RU'}, 'chat': {'id': 84723474, 'first_name': 'Александр', 'last_name': 'Толмачёв', 'type': 'private'}, 'date': 1502611993, 'text': '/help', 'entities': [{'type': 'bot_command', 'offset': 0, 'length': 5}]}}
X['message']['text'] = '/start'
X['message']['chat']['last_name']

# локальные тесты

In [1]:
from dialog_branch import *
import dialog_branch as dibr


chat_id = 1
json_update = {'update_id': 201011794, 'message': {'message_id': 159, 'from': {'id': 84723474, 'first_name': 'Александр', 'last_name': 'Толмачёв', 'language_code': 'ru-RU'}, 'chat': {'id': 84723474, 'first_name': 'Александр', 'last_name': 'Толмачёв', 'type': 'private'}, 'date': 1502611993, 'text': '/help', 'entities': [{'type': 'bot_command', 'offset': 0, 'length': 5}]}}


text , reply_markup  = text , reply_markup = dibr.start.welcome(chat_id = chat_id , json_update = json_update)

In [2]:
text

'🎉Добрый день! \nЯ веду домашний бюджет и напоминаю об операциях в течение месяца. \nЯ развиваюсь в свободное время, потом будет интереснее. \nСначала заведите свой кошелек и забудьте о том, чтобы держать бюджет семьи в голове!💵'

#  персональные новости

In [ ]:
import requests 

api_key = 'f45b069a9ab844369005986b34bb6764'
url = 'https://newsapi.org/v2/'

method = 'top-headlines?'

country = 'ru'
category_list = ['business', 'entertainment', 'general', 'health', 'science', 'sports', 'technology']
category = 'science'
language = 'ru'
from_parameter = '2018-08-02'
to_parameter = '2018-08-02'





get_url = url+method+'country='+country+'&category='+category+'&language='+language+'&from_parameter='+from_parameter+'&to_parameter='+to_parameter 
get_url += '&apiKey='+api_key

r = requests.get(get_url)

json_r_news = json.loads(r.text)

json_r_news['totalResults']

In [ ]:
json_r_news['articles'][0]